In [3]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import calendar
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [4]:
data=pd.read_csv('E:\Excelr-P269-Internship-Project-Group2-OIl-Price-Prediction-And-Forecasting-Using-Python\Dataset\IQR.csv')

In [4]:
df=pd.DataFrame(data)
print(df.tail())

            Date      Price  Year
5746  2023-07-17  74.150002  2023
5747  2023-07-18  75.750000  2023
5748  2023-07-19  75.349998  2023
5749  2023-07-20  75.629997  2023
5750  2023-07-21  77.070000  2023


In [5]:
import pandas as pd
from scipy.stats import boxcox
from sklearn.preprocessing import MinMaxScaler

data = data[['Price']]

# Fill missing values using interpolation
# Set the index to a DatetimeIndex
data.index = pd.DatetimeIndex(data.index)

# Fill missing values using interpolation
data = data.interpolate(method='time')
# Make the data stationary
def make_stationary(data):
    # Take the first difference of the data
    data_diff = data.diff().dropna()
    
    # Check if the data is stationary using the Augmented Dickey-Fuller test
    from statsmodels.tsa.stattools import adfuller
    result = adfuller(data_diff)
    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))
    
    # If the p-value is less than 0.05, the data is stationary
    if result[1] < 0.05:
        print('The data is stationary')
        return data_diff
    else:
        print('The data is not stationary')
        return make_stationary(data_diff)

data = make_stationary(data)

# Data Transformation
# Apply Box-Cox transformation
data['Price'] = data['Price'] - data['Price'].min() + 1
data['Price'], lam = boxcox(data['Price'])

# Data Normalization
# Normalize the data using MinMaxScaler
scaler = MinMaxScaler()
data['Price'] = scaler.fit_transform(data[['Price']])

# Feature Engineering
# Create lag features
for i in range(1, 13):
    data[f'lag_{i}'] = data['Price'].shift(i)

# Create rolling window statistics
data['rolling_mean'] = data['Price'].rolling(window=12).mean()
data['rolling_std'] = data['Price'].rolling(window=12).std()

# Train-Test Split
# Split the data into training and test sets
train_size = int(len(data) * 0.8)
train, test = data[0:train_size], data[train_size:len(data)]


ADF Statistic: -13.994867
p-value: 0.000000
Critical Values:
	1%: -3.431
	5%: -2.862
	10%: -2.567
The data is stationary


In [1]:
import pandas as pd
from fbprophet import Prophet
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error

# Load the data
data = pd.read_csv('data.csv', parse_dates=['Date'], index_col='Date')
data = data[['Price']]

# Prepare the data for Prophet
prophet_data = data.reset_index()
prophet_data.columns = ['ds', 'y']

# Split the data into training and test sets
train_size = int(len(data) * 0.8)
train, test = prophet_data[0:train_size], prophet_data[train_size:len(prophet_data)]

# Fit the Prophet model
prophet_model = Prophet()
prophet_model.fit(train)

# Make predictions using the Prophet model
prophet_forecast = prophet_model.predict(test[['ds']])
prophet_predictions = prophet_forecast['yhat'].values

# Calculate the mean squared error of the Prophet model
prophet_mse = mean_squared_error(test['y'], prophet_predictions)
print(f'Prophet MSE: {prophet_mse}')

# Prepare the data for LSTM
lstm_data = data.values

# Split the data into training and test sets
train, test = lstm_data[0:train_size], lstm_data[train_size:len(lstm_data)]

# Define a function to create input-output pairs for the LSTM model
def create_dataset(data, look_back=1):
    X, Y = [], []
    for i in range(len(data)-look_back-1):
        X.append(data[i:(i+look_back), 0])
        Y.append(data[i + look_back, 0])
    return np.array(X), np.array(Y)

# Create input-output pairs for the LSTM model
look_back = 1
X_train, y_train = create_dataset(train, look_back)
X_test, y_test = create_dataset(test, look_back)

# Reshape the data into [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Fit the LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(4, input_shape=(1, look_back)))
lstm_model.add(Dense(1))
lstm_model.compile(loss='mean_squared_error', optimizer='adam')
lstm_model.fit(X_train, y_train, epochs=100, batch_size=1)

# Make predictions using the LSTM model
lstm_predictions = lstm_model.predict(X_test)

# Calculate the mean squared error of the LSTM model
lstm_mse = mean_squared_error(y_test, lstm_predictions)
print(f'LSTM MSE: {lstm_mse}')

# Compare the performance of the two models
if prophet_mse < lstm_mse:
    print('Prophet performs better')
else:
    print('LSTM performs better')


ModuleNotFoundError: No module named 'fbprophet'

In [6]:
import pandas as pd
# from fbprophet import Prophet
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error

"""# Load the data
data = pd.read_csv('data.csv', parse_dates=['Date'], index_col='Date')


# Prepare the data for Prophet
prophet_data = data.reset_index()
prophet_data.columns = ['ds', 'y']

# Split the data into training and test sets
train_size = int(len(data) * 0.8)
train, test = prophet_data[0:train_size], prophet_data[train_size:len(prophet_data)]

# Fit the Prophet model
prophet_model = Prophet()
prophet_model.fit(train)

# Make predictions using the Prophet model
prophet_forecast = prophet_model.predict(test[['ds']])
prophet_predictions = prophet_forecast['yhat'].values

# Calculate the mean squared error of the Prophet model
prophet_mse = mean_squared_error(test['y'], prophet_predictions)
print(f'Prophet MSE: {prophet_mse}')"""

data = data[['Price']]
# Prepare the data for LSTM
lstm_data = data.values

# Split the data into training and test sets
train, test = lstm_data[0:train_size], lstm_data[train_size:len(lstm_data)]

# Define a function to create input-output pairs for the LSTM model
def create_dataset(data, look_back=1):
    X, Y = [], []
    for i in range(len(data)-look_back-1):
        X.append(data[i:(i+look_back), 0])
        Y.append(data[i + look_back, 0])
    return np.array(X), np.array(Y)

# Create input-output pairs for the LSTM model
look_back = 1
X_train, y_train = create_dataset(train, look_back)
X_test, y_test = create_dataset(test, look_back)

# Reshape the data into [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Fit the LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(4, input_shape=(1, look_back)))
lstm_model.add(Dense(1))
lstm_model.compile(loss='mean_squared_error', optimizer='adam')
lstm_model.fit(X_train, y_train, epochs=100, batch_size=1)

# Make predictions using the LSTM model
lstm_predictions = lstm_model.predict(X_test)

# Calculate the mean squared error of the LSTM model
lstm_mse = mean_squared_error(y_test, lstm_predictions)
print(f'LSTM MSE: {lstm_mse}')


Epoch 1/100
4598/4598 [==============================] - 14s 2ms/step - loss: 0.0046
Epoch 2/100
4598/4598 [==============================] - 12s 3ms/step - loss: 0.0019
Epoch 3/100
4598/4598 [==============================] - 13s 3ms/step - loss: 0.0019
Epoch 4/100
4598/4598 [==============================] - 14s 3ms/step - loss: 0.0019
Epoch 5/100
4598/4598 [==============================] - 13s 3ms/step - loss: 0.0019
Epoch 6/100
4598/4598 [==============================] - 13s 3ms/step - loss: 0.0019
Epoch 7/100
4598/4598 [==============================] - 12s 3ms/step - loss: 0.0019
Epoch 8/100
 415/4598 [=>............................] - ETA: 10s - loss: 0.0021